# Image Classification using LeNet CNN
## CIFAR-10 Dataset - 10 classes of animals and objects

In [ ]:
# import the necessary packages
from LeNet import LeNet
from sklearn.model_selection import train_test_split
from keras.datasets import cifar10
from keras.optimizers import RMSprop
from keras.utils import np_utils
from keras import backend as K
import numpy as np
import argparse
import cv2

## Load the data

In [ ]:
# grab the CIFAR-10 dataset (may take time the first time)
print("[INFO] downloading CIFAR-10...")
((trainData, trainLabels), (testData, testLabels)) = cifar10.load_data()

## Prepare the data

In [ ]:
# parameters for CIFAR-10 data set
num_classes = 10
image_width = 32
image_height = 32
image_channels = 3

In [ ]:
# shape the input data using "channels last" ordering
# num_samples x rows x columns x depth
trainData = trainData.reshape(
        (trainData.shape[0], image_height, image_width, image_channels))
testData = testData.reshape(
        (testData.shape[0], image_height, image_width, image_channels))

In [ ]:
# scale data to the range of [0.0, 1.0]
trainData = trainData.astype("float32") / 255.0
testData = testData.astype("float32") / 255.0

In [ ]:
# transform the training and testing labels into vectors in the
# range [0, classes] -- this generates a vector for each label,
# where the index of the label is set to `1` and all other entries
# to `0`; in the case of CIFAR-10, there are 10 class labels
trainLabels = np_utils.to_categorical(trainLabels, num_classes)  # one hot encoding
testLabels = np_utils.to_categorical(testLabels, num_classes)

## Train Model

In [ ]:
# initialize the model
print("[INFO] compiling model...")
model = LeNet.build(numChannels=image_channels, 
                    imgRows=image_height, imgCols=image_width,
                    numClasses=num_classes,
                    weightsPath=None)

# initialize the optimizer
opt = RMSprop(lr=0.0001, decay=1e-6)  # RMS Prop

# build the model
model.compile(loss="categorical_crossentropy", # Soft-Max
              optimizer=opt, metrics=["accuracy"])

In [ ]:
# initialize hyper parameters
batch_size = 128
epochs = 1

print("[INFO] training...")
model.fit(trainData, trainLabels, batch_size=batch_size,
          epochs=epochs, verbose=1)

In [ ]:
# show the accuracy on the testing set
print("[INFO] evaluating...")
(loss, accuracy) = model.evaluate(testData, testLabels,
                                  batch_size=batch_size, verbose=1)
print("[INFO] accuracy: {:.2f}%".format(accuracy * 100))

In [ ]:
model.save_weights("lenet_cifar_test.hdf5", overwrite=True)

## Evaluate Pre-trained Model

In [ ]:
# load the model weights
print("[INFO] compiling model...")
model = LeNet.build(numChannels=image_channels, 
                    imgRows=image_height, imgCols=image_width,
                    numClasses=num_classes,
                    weightsPath="weights/lenet_weights_cifar.hdf5")

# initialize the optimizer
opt = RMSprop(lr=0.0001, decay=1e-6)  # RMS Prop

# build the model
model.compile(loss="categorical_crossentropy", # Soft-Max
              optimizer=opt, metrics=["accuracy"])

In [ ]:
# show the accuracy on the testing set
print("[INFO] evaluating...")
(loss, accuracy) = model.evaluate(testData, testLabels,
                                  batch_size=batch_size, verbose=1)
print("[INFO] accuracy: {:.2f}%".format(accuracy * 100))

## Model Predictions

In [ ]:
# set prediction parameters
num_predictions = 10

# randomly select a few testing digits
for i in np.random.choice(np.arange(0, len(testLabels)), size=(num_predictions,)):
    # classify the digit
    probs = model.predict(testData[np.newaxis, i])
    prediction = probs.argmax(axis=1)

    # extract the image from the testData
    chR = (testData[i][:, :, 0] * 255).astype("uint8")
    chG = (testData[i][:, :, 1] * 255).astype("uint8")
    chB = (testData[i][:, :, 2] * 255).astype("uint8")

    # merge the channels into one image
    image = cv2.merge((chB, chG, chR))

    # resize the image from a 32 x 32 image to a 96 x 96 image so we can better see it
    image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)

    print("[INFO] Predicted: {}, Actual: {}".format(
        prediction[0], np.argmax(testLabels[i])))

    # show the image and prediction
    classLabels = ['airplane', 'automobile', 'bird', 'cat',
                   'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    cv2.putText(image, classLabels[prediction[0]], (5, 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
    cv2.imshow("Object", image)
    cv2.waitKey(0)
    
# close the display window
cv2.destroyAllWindows()